In [8]:
# Minimal 1-bus PyPSA example: expandable generator + fixed-size storage + expensive slack + load
import pypsa
import numpy as np
import pandas as pd

# --- Time span ---
snapshots = pd.date_range("2025-01-01", periods=24, freq="h")

# --- Network ---
n = pypsa.Network()
n.set_snapshots(snapshots)

# --- Bus ---
n.add("Bus", "bus0")

# --- Carriers ---
n.add("Carrier", "VRE")
n.add("Carrier", "slack")
n.add("Carrier", "battery")

# --- Load profile (MW): base + small variation ---
base_load = 100.0
daily_shape = 1.0 + 0.2*np.sin(2*np.pi*(snapshots.hour-8)/24)
p_set = base_load * daily_shape
n.add("Load", "load0", bus="bus0", p_set=pd.Series(p_set, index=snapshots))

# --- Variable generator (expandable) ---
solar_like = np.clip(np.sin(np.pi*(snapshots.hour-6)/12), 0, 1)
n.add(
    "Generator",
    "gen_var",
    bus="bus0",
    carrier="VRE",
    p_nom_extendable=True,     
    p_nom_min=0.0,
    p_max_pu=pd.Series(solar_like, index=snapshots),
    capital_cost=40.0,          
    marginal_cost=0.0,
)

# --- StorageUnit (fixed size, NOT expandable) ---
n.add(
    "StorageUnit",
    "storage0",
    bus="bus0",
    carrier="battery",
    p_nom=30.0,                
    max_hours=3.0,              
    p_nom_extendable=False,     
    efficiency_store=0.95,
    efficiency_dispatch=0.95,
    standing_loss=0.0,
    cyclic_state_of_charge=True,
    marginal_cost=0.0,
    capital_cost=1500.0,          
)

# --- Slack unit (very expensive) ---
n.add(
    "Generator",
    "slack",
    bus="bus0",
    carrier="slack",
    p_nom=1e6,                
    p_max_pu=1.0,
    marginal_cost=1e4,         
)

n.optimize(solver_name="gurobi")


Index(['bus0'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.13s


Set parameter Username


INFO:gurobipy:Set parameter Username


Set parameter LicenseID to value 2647006


INFO:gurobipy:Set parameter LicenseID to value 2647006


...

Solved in 13 iterations and 0.01 seconds (0.00 work units)


INFO:gurobipy:Solved in 13 iterations and 0.01 seconds (0.00 work units)


Optimal objective  1.084701725e+07


INFO:gurobipy:Optimal objective  1.084701725e+07
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 121 primals, 289 duals
Objective: 1.08e+07
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

In [16]:
stats = (n.statistics.capex().sum() + n.statistics.opex().sum()) / 1e6
obj = (n.objective + n.objective_constant) / 1e6
obj_const = n.objective_constant / 1e6
non_ext_cost = (n.storage_units.p_nom * n.storage_units.capital_cost).sum() / 1e6

stats - obj - (non_ext_cost - obj_const)  # should be zero

7.339837071462796e-12

In [9]:
(n.statistics.capex().sum() + n.statistics.opex().sum()) / 1e6

10.89201724937

In [10]:
(n.objective + n.objective_constant) / 1e6

10.84701724936266

In [11]:
n.objective_constant

0.0

In [15]:
(n.storage_units.p_nom * n.storage_units.capital_cost).sum() / 1e6

0.045

In [4]:
# network.statistics()
(network.statistics()['Capital Expenditure'].sum() + network.statistics()['Operational Expenditure'].sum()) / 1e8 # - (network.objective + network.objective_constant) / 1e8 + network.objective_constant / 1e8 - (19711.86918 * 55 + 100 * 251621.03	+ 100 * 132325.72 + 100 * 222.49) / 1e8

NameError: name 'network' is not defined

In [79]:
(network.statistics.capex().sum() + network.statistics.opex().sum()) / 1e8

4.1316465973424

In [80]:
network.statistics.system_cost().sum() / 1e8

4.1316465973424

In [81]:
(network.objective + network.objective_constant) / 1e8

4.131424107342484

In [82]:
network.objective_constant / 1e6

1.0841528051052216

In [83]:
(19711.86918 * 55 + 100 * 251621.03	+ 100 * 132325.72 + 100 * 222.49) / 1e6

39.5010768049

In [84]:
if transformation.dimensions['InvestmentBlock']['NumAssets'] == 0 or transformation.expansion_ucblock:
    ### UCBlock configuration ###
    configfile = pysmspp.SMSConfig(template="UCBlock/uc_solverconfig_grb")  # load a default config file [highs solver]
    temporary_smspp_file = "output/network_uc_hydro_0011.nc"  # path to temporary SMS++ file
    output_file = "output/temp_log_file.txt"  # path to the output file (optional)
    solution_file = "output/solution_uc_hydro_0011.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, log_executable_call=True)
    
    statistics = network.statistics()
    operational_cost = statistics['Operational Expenditure'].sum()
    # error = (operational_cost - result.objective_value) / operational_cost * 100

    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    
    # Esegui la funzione sul file di testo
    data_dict = parse_txt_file(output_file)

    print(f"Il solver ci ha messo {data_dict['elapsed_time']}s")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    
    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    # transformation.parse_txt_to_unitblocks(output_file)
    transformation.inverse_transformation(nd.n)

    differences = compare_networks(network, nd.n)
    statistics_smspp = nd.n.statistics()
    

else:
    ### InvestmentBlock configuration ###
    configfile = pysmspp.SMSConfig(template="InvestmentBlock/BSPar.txt")
    temporary_smspp_file = "output/temp_network_investment_daily.nc"
    output_file = "output/temp_log_file_investment.txt"  # path to the output file (optional)
    solution_file = "output/temp_solution_file_investment.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, inner_block_name='InvestmentBlock', log_executable_call=True)
    
    
    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    transformation.inverse_transformation(nd.n)

OverflowError: can't convert negative value to size_t

In [ ]:
nd.n.statistics()
# (nd.n.statistics()['Capital Expenditure'].sum() + nd.n.statistics()['Operational Expenditure'].sum()) / 1e8

Optimal Capacity  Installed Capacity       Supply  \
Generator slack                  6232.15145          6232.15145  40047.54549   
          solar                 74626.39113         19711.86918  27977.51015   
Link      H2 electrolysis         100.00000           100.00000      0.00000   
          H2 fuel cell            100.00000           100.00000      0.00000   
Load      AC                        0.00000             0.00000      0.00000   
Store     H2                      100.00000           100.00000    348.00000   

                            Withdrawal  Energy Balance Transmission  \
Generator slack                0.00000     40047.54549          0.0   
          solar                0.00000     27977.51015          0.0   
Link      H2 electrolysis   1200.00000     -1200.00000          0.0   
          H2 fuel cell       348.00000      -348.00000          0.0   
Load      AC               67173.05565    -67173.05565          0.0   
Store     H2                1200.00000      -852.00000          0.0   

                           Capacity Factor   Curtailment  Capital Expenditure  \
Generator slack                   0.267748  109524.08924         0.000000e+00   
          solar                   0.015621  451571.67925         4.104452e+06   
Link      H2 electrolysis         0.500000       0.00000         2.516210e+07   
          H2 fuel cell            0.145000       0.00000         1.323257e+07   
Load      AC                           NaN       0.00000         0.000000e+00   
Store     H2                      2.966667       0.00000         2.224900e+04   

                           Operational Expenditure  Revenue  Market Value  
Generator slack                       4.004755e+08      0.0           NaN  
          solar                       2.797751e+02      0.0           NaN  
Link      H2 electrolysis             0.000000e+00      0.0           0.0  
          H2 fuel cell                0.000000e+00      0.0           0.0  
Load      AC                          0.000000e+00      0.0           0.0  
Store     H2                          0.000000e+00      0.0           NaN

In [ ]:
# network.generators[['capital_cost', 'marginal_cost']]
# network.links[['capital_cost', 'marginal_cost']]
network.stores[['capital_cost', 'marginal_cost']]

,capital_cost,marginal_cost
name,,
IT0 0 H2,222.49,0.0


In [ ]:
import pandas as pd
pd.DataFrame({
    "PyPSA e": network.stores_t["e"]['IT0 0 H2'],
    "PyPSA p": network.stores_t["p"]['IT0 0 H2'],
    "PyPSA solar": network.generators_t['p']['IT0 0 0 solar'],
    "PyPSA slack": network.generators_t['p']['slack_unit IT0 0'],
    "SMS++ e": nd.n.stores_t["e"]['IT0 0 H2'],
    "SMS++ p": nd.n.stores_t["p"]['IT0 0 H2'],
    "SMS++ solar": nd.n.generators_t['p']['IT0 0 0 solar'],
    "SMS++ slack": nd.n.generators_t['p']['slack_unit IT0 0']
})

,PyPSA e,PyPSA p,PyPSA solar,PyPSA slack,SMS++ e,SMS++ p,SMS++ solar,SMS++ slack
snapshot,,,,,,,,
0,0.0,0.0,0.000000,1738.142868,0.0,0.0,0.000000,1738.142868
1,0.0,0.0,0.000000,1781.508989,0.0,0.0,0.000000,1781.508989
2,0.0,0.0,0.000000,1536.154362,0.0,0.0,0.000000,1536.154362
3,0.0,0.0,0.000000,1522.320559,0.0,0.0,0.000000,1522.320559
4,0.0,0.0,1292.454633,0.000000,58.0,-100.0,1392.454633,0.000000
5,0.0,0.0,1215.634604,0.000000,116.0,-100.0,1315.634604,0.000000
6,42.0,-42.0,1144.754965,0.000000,174.0,-100.0,1172.341172,0.000000
7,100.0,-58.0,2764.640314,0.000000,232.0,-100.0,2764.640314,0.000000
8,100.0,0.0,3080.587260,0.000000,290.0,-100.0,3180.587260,0.000000


In [ ]:
transformation.unitblocks['BatteryUnitBlock_2']['variables']['MaxStorage']

{'value': array([[100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.],
        [100.]]),
 'type': 'float',
 'size': ('TimeHorizon',)}

In [ ]:
transformation.unitblocks['IntermittentUnitBlock_0']['variables']

{'Gamma': {'value': 0.0, 'type': 'float', 'size': ()},
 'Kappa': {'value': 1.0, 'type': 'float', 'size': ()},
 'MaxPower': {'value': array([[0.   ],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.213],
         [0.436],
         [0.593],
         [0.7  ],
         [0.764],
         [0.786],
         [0.772],
         [0.72 ],
         [0.627],
         [0.49 ],
         [0.288],
         [0.037],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.   ],
         [0.   ]]),
  'type': 'float',
  'size': ('TimeHorizon',)},
 'MinPower': {'value': array([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]),
  't

In [ ]:
network.links.efficiency

name
IT0 0 H2 Electrolysis    0.58
IT0 0 H2 Fuel Cell       0.50
Name: efficiency, dtype: float64

In [ ]:
import pandas as pd
df1 = nd.n.stores_t['p']      # primo DataFrame
df2 = network.stores_t['p']   # secondo DataFrame
df1.join(df2, how='inner', lsuffix='_nd', rsuffix='_net')

name,IT0 0 H2_nd,IT0 0 H2_net
snapshot,,
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,-100.0,0.0
5,-100.0,0.0
6,-100.0,-42.0
7,-100.0,-58.0
8,-100.0,0.0


In [ ]:
# --- Multi-column case (align on common link names) ---
p0_nd  = nd.n.links_t['p0']
p0_net = network.links_t['p0']
p1_nd  = nd.n.links_t['p1']
p1_net = network.links_t['p1']

common_links_p0 = p0_nd.columns.intersection(p0_net.columns)
common_links_p1 = p1_nd.columns.intersection(p1_net.columns)

p0_cmp = p0_nd[common_links_p0].join(p0_net[common_links_p0], lsuffix='_nd', rsuffix='_net')
p1_cmp = p1_nd[common_links_p1].join(p1_net[common_links_p1], lsuffix='_nd', rsuffix='_net')

# Pack everything together with a neat column MultiIndex: ('p0','battery_nd'), ('p0','battery_net'), ...
links_merged = pd.concat({'p0': p0_cmp, 'p1': p1_cmp}, axis=1)

# Diffs for common links
p0_diff = p0_nd[common_links_p0] - p0_net[common_links_p0]
p1_diff = p1_nd[common_links_p1] - p1_net[common_links_p1]

links_merged

p0                        \
name     IT0 0 H2 Electrolysis_nd IT0 0 H2 Fuel Cell_nd   
snapshot                                                  
0                             0.0                   0.0   
1                             0.0                   0.0   
2                             0.0                   0.0   
3                             0.0                   0.0   
4                           100.0                   0.0   
5                           100.0                   0.0   
6                           100.0                   0.0   
7                           100.0                   0.0   
8                           100.0                   0.0   
9                           100.0                   0.0   
10                          100.0                   0.0   
11                          100.0                   0.0   
12                          100.0                   0.0   
13                          100.0                   0.0   
14                          100.0                   0.0   
15                          100.0                   0.0   
16                            0.0                  50.0   
17                            0.0                  48.0   
18                            0.0                   0.0   
19                            0.0                  50.0   
20                            0.0                  50.0   
21                            0.0                  50.0   
22                            0.0                  50.0   
23                            0.0                  50.0   

                                                           \
name     IT0 0 H2 Electrolysis_net IT0 0 H2 Fuel Cell_net   
snapshot                                                    
0                         0.000000                    0.0   
1                         0.000000                    0.0   
2                         0.000000                    0.0   
3                         0.000000                    0.0   
4                         0.000000                    0.0   
5                         0.000000                    0.0   
6                        72.413793                    0.0   
7                       100.000000                    0.0   
8                         0.000000                    0.0   
9                         0.000000                    0.0   
10                        0.000000                    0.0   
11                        0.000000                    0.0   
12                        0.000000                    0.0   
13                        0.000000                    0.0   
14                        0.000000                    0.0   
15                        0.000000                    0.0   
16                        0.000000                    0.0   
17                        0.000000                  100.0   
18                        0.000000                    0.0   
19                        0.000000                    0.0   
20                        0.000000                    0.0   
21                        0.000000                    0.0   
22                        0.000000                    0.0   
23                        0.000000                    0.0   

                               p1                        \
name     IT0 0 H2 Electrolysis_nd IT0 0 H2 Fuel Cell_nd   
snapshot                                                  
0                            -0.0                  -0.0   
1                            -0.0                  -0.0   
2                            -0.0                  -0.0   
3                            -0.0                  -0.0   
4                           -58.0                  -0.0   
5                           -58.0                  -0.0   
6                           -58.0                  -0.0   
7                           -58.0                  -0.0   
8                           -58.0                  -0.0   
9                           -58.0                  -0.0   
10                    

In [ ]:
# Take the four series/single-col DF and give them clear names
p_nd  = nd.n.stores_t['p'].squeeze().rename('p_smspp')   # net power (nd)
p_net = network.stores_t['p'].squeeze().rename('p_pypsa')

e_nd  = nd.n.stores_t['e'].squeeze().rename('e_smspp')         # energy/SoC (nd)
e_net = network.stores_t['e'].squeeze().rename('e_pypsa')

# Optional sanity check: same index (snapshots)
assert p_nd.index.equals(p_net.index) and p_nd.index.equals(e_nd.index) and p_nd.index.equals(e_net.index), \
    "Indices must match"

# Simple side-by-side merge
pd.concat([p_nd, p_net, e_nd, e_net], axis=1)

# Now `merged` has columns: ['p_nd', 'p_net', 'e_nd', 'e_net']

,p_smspp,p_pypsa,e_smspp,e_pypsa
snapshot,,,,
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,-100.0,0.0,58.0,0.0
5,-100.0,0.0,116.0,0.0
6,-100.0,-42.0,174.0,42.0
7,-100.0,-58.0,232.0,100.0
8,-100.0,0.0,290.0,100.0


In [ ]:
df1 = nd.n.generators_t['p']      # primo DataFrame
df2 = network.loads_t['p']   # secondo DataFrame
df1.join(df2, how='inner', lsuffix='_nd', rsuffix='_net')

name,IT0 0 0 solar,slack_unit IT0 0,IT0 0
snapshot,,,
0,0.000000,1738.142868,1738.142868
1,0.000000,1781.508989,1781.508989
2,0.000000,1536.154362,1536.154362
3,0.000000,1522.320559,1522.320559
4,1392.454633,0.000000,1292.454633
5,1315.634604,0.000000,1215.634604
6,1172.341172,0.000000,1072.341172
7,2764.640314,0.000000,2664.640314
8,3180.587260,0.000000,3080.587260


In [ ]:
transformation.unitblocks['DCNetworkBlock_3']

{'enumerate': 'UnitBlock_3',
 'block': 'DCNetworkBlock_links',
 'name': 'IT0 0 H2 Electrolysis',
 'FlowValue': masked_array(data=[  0.,   0.,   0.,   0., 100., 100., 100., 100., 100.,
                    100., 100., 100., 100., 100., 100., 100.,   0.,   0.,
                      0.,   0.,   0.,   0.,   0.,   0.],
              mask=False,
        fill_value=1e+20),
 'DualCost': masked_array(data=[  -0.        ,   -0.        ,   -0.        ,
                      -0.        , 2899.99      , 2899.99      ,
                    2899.99      , 2899.99      , 2899.99      ,
                    2899.99      , 2899.99      , 2899.99      ,
                    2899.99      , 2899.99      , 2899.99      ,
                    1413.50351351,   -0.        ,   -0.        ,
                      -0.        ,   -0.        ,   -0.        ,
                      -0.        ,   -0.        ,   -0.        ],
              mask=False,
        fill_value=1e+20),
 'DesignVariable': 100.0,
 'Efficiencies': [1.

In [ ]:
transformation.unitblocks['IntermittentUnitBlock_0']

{'name': 'IT0 0 0 solar',
 'enumerate': 'UnitBlock_0',
 'block': 'IntermittentUnitBlock',
 'IntermittentDesign': masked_array(data=74626.39113006,
              mask=False,
        fill_value=1e+20),
 'Extendable': True,
 'variables': {'Gamma': {'value': 0.0, 'type': 'float', 'size': ()},
  'Kappa': {'value': 1.0, 'type': 'float', 'size': ()},
  'MaxPower': {'value': array([[0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.213],
          [0.436],
          [0.593],
          [0.7  ],
          [0.764],
          [0.786],
          [0.772],
          [0.72 ],
          [0.627],
          [0.49 ],
          [0.288],
          [0.037],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ]]),
   'type': 'float',
   'size': ('TimeHorizon',)},
  'MinPower': {'value': array([[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
        